In [32]:
file_path = 'constitutionrf.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [33]:
from langchain_text_splitters import CharacterTextSplitter
from pprint import pprint

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    #encoding=get_encoding('cl100k_base'),  # В новых версия LangChain эту опция более недоступна
    chunk_size=200,
    chunk_overlap=20
)
chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
pprint(chunks[:10])

Created a chunk of size 787, which is longer than the specified 200
Created a chunk of size 514, which is longer than the specified 200
Created a chunk of size 301, which is longer than the specified 200
Created a chunk of size 919, which is longer than the specified 200
Created a chunk of size 461, which is longer than the specified 200
Created a chunk of size 557, which is longer than the specified 200
Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 315, which is longer than the specified 200
Created a chunk of size 214, which is longer than the specified 200
Created a chunk of size 675, which is longer than the specified 200
Created a chunk of size 237, which is longer than the specified 200
Created a chunk of size 674, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 1078, which is longer than the specified 200
Created a chunk of size 333, which is longer th

Всего чанков: 273
Первые N чанков:
['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\n'
 'соединенные общей судьбой на своей земле,\n'
 'утверждая права и свободы человека, гражданский мир и согласие,\n'
 'сохраняя исторически сложившееся государственное единство,\n'
 'исходя из общепризнанных принципов равноправия и самоопределения народов,\n'
 'чтя память предков, передавших нам любовь и уважение к Отечеству, веру в '
 'добро и справедливость,\n'
 'возрождая суверенную государственность России и утверждая незыблемость ее '
 'демократической основы,\n'
 'стремясь обеспечить благополучие и процветание России,\n'
 'исходя из ответственности за свою Родину перед нынешним и будущими '
 'поколениями,\n'
 'сознавая себя частью мирового сообщества,\n'
 'принимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n\nГЛАВА 1.\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ\n\nСтатья 1',
 'Статья 1\n'
 '\n'
 '1. Российская Федерация - Россия есть демократическое фе

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=200,
    chunk_overlap=20
)

chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
pprint(chunks[:10])

Всего чанков: 463
Первые N чанков:
['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\n'
 'соединенные общей судьбой на своей земле,\n'
 'утверждая права и свободы человека, гражданский мир и согласие,\n'
 'сохраняя исторически сложившееся государственное единство,\n'
 'исходя из общепризнанных принципов равноправия и самоопределения народов,\n'
 'чтя память предков, передавших нам любовь и уважение к Отечеству, веру в '
 'добро и справедливость,',
 'возрождая суверенную государственность России и утверждая незыблемость ее '
 'демократической основы,\n'
 'стремясь обеспечить благополучие и процветание России,\n'
 'исходя из ответственности за свою Родину перед нынешним и будущими '
 'поколениями,\n'
 'сознавая себя частью мирового сообщества,\n'
 'принимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n'
 '\n'
 'ГЛАВА 1.\n'
 'ОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ\n'
 '\n'
 'Статья 1\n'
 '\n'
 '1. Российская Федерация - Россия есть демократичес

In [35]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

splitter = SentenceTransformersTokenTextSplitter(
    model_name="sentence-transformers/all-mpnet-base-v2",
    chunk_size=200, chunk_overlap=20
)
text_token_count = splitter.count_tokens(text=text) - 2
token_multiplier = splitter.maximum_tokens_per_chunk // text_token_count + 1
text_to_split = text * token_multiplier  # Пример использования для нарезки большого текста
print(f"Токенов в тексте для нарезки: {splitter.count_tokens(text=text_to_split)}")

chunks = splitter.split_text(text=text_to_split)
print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
pprint(chunks[:10])

Токенов в тексте для нарезки: 105819
Всего чанков: 291
Первые N чанков:
['конституция россиискои федерации мы, многонациональныи народ россиискои '
 'федерации, соединенные общеи судьбои на своеи земле, утверждая права и '
 'свободы человека, гражданскии мир и согласие, сохраняя исторически '
 'сложившееся государственное единство, исходя из общепризнанных принципов '
 'равноправия и самоопределения народов, чтя память предков, передавших нам '
 'любовь и уважение к отечеству, веру в добро и справедливость, возрождая '
 'суверенную государственност',
 '##нную государственность россии и утверждая незыблемость ее демократическои '
 'основы, стремясь обеспечить благополучие и процветание россии, исходя из '
 'ответственности за свою родину перед нынешним и будущими поколениями, '
 'сознавая себя частью мирового сообщества, принимаем конституцию россиискои '
 'федерации. раздел первыи глава 1. основы конституционного строя статья 1 1. '
 'россииская федерация - россия есть демократическое 

In [36]:
from transformers import GPT2TokenizerFast
from langchain_text_splitters import CharacterTextSplitter

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)
chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
pprint(chunks[:10])

Created a chunk of size 787, which is longer than the specified 100
Created a chunk of size 191, which is longer than the specified 100
Created a chunk of size 162, which is longer than the specified 100
Created a chunk of size 514, which is longer than the specified 100
Created a chunk of size 301, which is longer than the specified 100
Created a chunk of size 919, which is longer than the specified 100
Created a chunk of size 461, which is longer than the specified 100
Created a chunk of size 557, which is longer than the specified 100
Created a chunk of size 354, which is longer than the specified 100
Created a chunk of size 315, which is longer than the specified 100
Created a chunk of size 214, which is longer than the specified 100
Created a chunk of size 675, which is longer than the specified 100
Created a chunk of size 237, which is longer than the specified 100
Created a chunk of size 674, which is longer than the specified 100
Created a chunk of size 215, which is longer tha

Всего чанков: 293
Первые N чанков:
['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\n'
 'соединенные общей судьбой на своей земле,\n'
 'утверждая права и свободы человека, гражданский мир и согласие,\n'
 'сохраняя исторически сложившееся государственное единство,\n'
 'исходя из общепризнанных принципов равноправия и самоопределения народов,\n'
 'чтя память предков, передавших нам любовь и уважение к Отечеству, веру в '
 'добро и справедливость,\n'
 'возрождая суверенную государственность России и утверждая незыблемость ее '
 'демократической основы,\n'
 'стремясь обеспечить благополучие и процветание России,\n'
 'исходя из ответственности за свою Родину перед нынешним и будущими '
 'поколениями,\n'
 'сознавая себя частью мирового сообщества,\n'
 'принимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n\nГЛАВА 1.\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ',
 'Статья 1',
 '1. Российская Федерация - Россия есть демократическое федеративное правовое